In [2]:

import pandas as pd
import re
import os

def parse_training_results(file_path):
    """Parse training results from text file into a pandas DataFrame"""
    
    # Read the file
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Initialize lists to store data
    epochs = []
    times = []
    train_losses = []
    train_acc_top1 = []
    train_acc_top5 = []
    test_losses = []
    test_acc_top1 = []
    test_acc_top5 = []
    
    # Regular expression to match each epoch line
    pattern = r'\[Epoch (\d+)\] Time: ([\d.]+)s.*?Train.*?Loss: ([\d.]+).*?Top1: ([\d.]+)%.*?Top5: ([\d.]+)%.*?Test.*?Loss: ([\d.]+).*?Top1: ([\d.]+)%.*?Top5: ([\d.]+)%'
    
    # Find all matches
    matches = re.findall(pattern, content)
    
    for match in matches:
        epochs.append(int(match[0]))
        times.append(float(match[1]))
        train_losses.append(float(match[2]))
        train_acc_top1.append(float(match[3]))
        train_acc_top5.append(float(match[4]))
        test_losses.append(float(match[5]))
        test_acc_top1.append(float(match[6]))
        test_acc_top5.append(float(match[7]))
    
    # Create DataFrame
    df = pd.DataFrame({
        'epoch': epochs,
        'time': times,
        'train_loss': train_losses,
        'train_accuracy_top1': train_acc_top1,
        'train_accuracy_top5': train_acc_top5,
        'test_loss': test_losses,
        'test_accuracy_top1': test_acc_top1,
        'test_accuracy_top5': test_acc_top5
    })
    
    return df


def n_last_average_stats(df, last_n_epochs=5):
    """Calculate average stats over the last N epochs"""
    if len(df) < last_n_epochs:
        last_n_epochs = len(df)
    
    return {
        "ave_train_loss": df['train_loss'].tail(last_n_epochs).mean(),
        "ave_test_loss": df['test_loss'].tail(last_n_epochs).mean(),
        "ave_train_accuracy_top1": df['train_accuracy_top1'].tail(last_n_epochs).mean(),
        "ave_train_accuracy_top5": df['train_accuracy_top5'].tail(last_n_epochs).mean(),
        "ave_test_accuracy_top1": df['test_accuracy_top1'].tail(last_n_epochs).mean(),
        "ave_test_accuracy_top5": df['test_accuracy_top5'].tail(last_n_epochs).mean()
    }

def best_stats(df):
    """Calculate best accuracy achieved during training"""
    return {
        "best_train_accuracy_top1": df['train_accuracy_top1'].max(),
        "best_train_accuracy_top5": df['train_accuracy_top5'].max(),
        "best_test_accuracy_top1": df['test_accuracy_top1'].max(),
        "best_test_accuracy_top5": df['test_accuracy_top5'].max()
    }

def last_epoch_stats(df):
    """Get stats from the last epoch"""
    last_row = df.iloc[-1]
    return {
        "last_epoch_train_loss": last_row['train_loss'],
        "last_epoch_test_loss": last_row['test_loss'],
        "last_epoch_train_accuracy_top1": last_row['train_accuracy_top1'],
        "last_epoch_train_accuracy_top5": last_row['train_accuracy_top5'],
        "last_epoch_test_accuracy_top1": last_row['test_accuracy_top1'],
        "last_epoch_test_accuracy_top5": last_row['test_accuracy_top5']
    }

def parse_params_gflops(file_dir):
    """Parse training results from text file into a pandas DataFrame"""

    train_path = os.path.join(file_dir, "train_eval_results.txt")
    args_path = os.path.join(file_dir, "args.txt")
    
    # Read the file
    with open(train_path, 'r') as f:
        train_content = f.readline()

    with open(args_path, 'r') as f:
        args_content = f.readlines()

    total_params = args_content[-2].strip().split(": ")[1]
    trainable_params = args_content[-1].strip().split(": ")[1]

    gflops = re.search(r'GFLOPs: ([\d.]+)', train_content).group(1)

    return {
        "total_params": int(total_params),
        "trainable_params": int(trainable_params),
        "gflops": float(gflops)
    }



In [3]:
import sys 
import os
import torch 
import torch.nn as nn

curr_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(curr_dir, '..', '..'))

print("Current Directory:", curr_dir)
print("Root Directory:", root_dir)


Current Directory: /Users/mingikang/Developer/ZiLU-Activation/Output
Root Directory: /Users/mingikang/Developer


In [4]:
save_path =  os.path.join(curr_dir, "csv")
print("Save Path:", save_path)
print(curr_dir)

results_path = os.path.join(curr_dir, "AUG/ResNet34/")
print("Results Path:", results_path)

Save Path: /Users/mingikang/Developer/ZiLU-Activation/Output/csv
/Users/mingikang/Developer/ZiLU-Activation/Output
Results Path: /Users/mingikang/Developer/ZiLU-Activation/Output/AUG/ResNet34/


In [ ]:
# 
datasets = ["CIFAR10", "CIFAR100"]
activations = ["zilu"]
# activations = ["zilu", "zilu_approx", "gelu_s", "silu_s"]
alphas = ["0.01", "0.05", "0.1", "0.5", "1.0", "5.0", "10.0", "50.0", "100.0", "1000.0"]


rows = [] 
for ds in datasets: 
    for act in activations:
        alpha_test_loss = {} 
        alpha_train_loss = {}

        save_path_test = os.path.join(curr_dir, "csv/ResNet34/alphas/", f"{ds}_{act}_test_loss.csv")
        save_path_train = os.path.join(curr_dir, "csv/ResNet34/alphas/", f"{ds}_{act}_train_loss.csv")

        for alpha in alphas:
            dir_path = os.path.join(results_path, ds, f"{act}_sigma{alpha}_s42")
            txt_path = os.path.join(dir_path, "train_eval_results.txt")

            print(f"Processing {txt_path} and saving to {save_path_test} and {save_path_train}")

            df = parse_training_results(txt_path)

            test_loss = df["test_loss"].tolist() 
            train_loss = df["train_loss"].tolist()


            alpha_test_loss[f"{act}_sigma{alpha}_test_loss"] = test_loss
            alpha_train_loss[f"{act}_sigma{alpha}_train_loss"] = train_loss
            
        
        df_train = pd.DataFrame(alpha_train_loss)
        df_train.insert(0, "epoch", list(range(1, len(df_train)+1)))
        df_train.to_csv(save_path_train, index=False)

        df_test = pd.DataFrame(alpha_test_loss)
        df_test.insert(0, "epoch", list(range(1, len(df_test)+1)))
        df_test.to_csv(save_path_test, index=False)
    


Processing /Users/mingikang/Developer/ZiLU-Activation/Output/AUG/ResNet34/CIFAR10/zilu_sigma0.01_s42/train_eval_results.txt and saving to /Users/mingikang/Developer/ZiLU-Activation/Output/csv/ResNet34/alphas/CIFAR10_zilu_test_loss.csv and /Users/mingikang/Developer/ZiLU-Activation/Output/csv/ResNet34/alphas/CIFAR10_zilu_train_loss.csv
Processing /Users/mingikang/Developer/ZiLU-Activation/Output/AUG/ResNet34/CIFAR10/zilu_sigma0.05_s42/train_eval_results.txt and saving to /Users/mingikang/Developer/ZiLU-Activation/Output/csv/ResNet34/alphas/CIFAR10_zilu_test_loss.csv and /Users/mingikang/Developer/ZiLU-Activation/Output/csv/ResNet34/alphas/CIFAR10_zilu_train_loss.csv
Processing /Users/mingikang/Developer/ZiLU-Activation/Output/AUG/ResNet34/CIFAR10/zilu_sigma0.1_s42/train_eval_results.txt and saving to /Users/mingikang/Developer/ZiLU-Activation/Output/csv/ResNet34/alphas/CIFAR10_zilu_test_loss.csv and /Users/mingikang/Developer/ZiLU-Activation/Output/csv/ResNet34/alphas/CIFAR10_zilu_train

In [30]:
df.head()

,epoch,time,train_loss,train_accuracy_top1,train_accuracy_top5,test_loss,test_accuracy_top1,test_accuracy_top5
0,1,75.0541,4.154967,6.6892,22.9907,3.944422,9.5431,32.1994
1,2,13.2142,3.629228,14.1796,38.9682,3.806319,18.9676,46.5487
2,3,13.3290,3.232599,20.9922,49.9480,3.028036,25.9197,56.2302
3,4,13.3888,2.944658,26.2612,57.4552,3.056888,28.9260,59.4442
4,5,13.2933,2.760256,29.8537,61.5213,2.707575,31.5269,63.4593
